In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from models.darcy import cc1
from models.darcy import cc2
from models.darcy import cc2new
from models.darcy import bc

from models.darcy import cc1_new, bc_new

import numpy as np
from utils.image_gradient import SobelFilter
from FEA_simp import ComputeTarget

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# random data
input = np.array(np.random.random([1,1,20,40]), dtype=np.float32)
# ComputeTarget是在FEA_simp.py中封装的计算真值的函数
output = torch.from_numpy(np.array(ComputeTarget(input), dtype=np.float32)).to(device)
input = torch.from_numpy(input).to(device)

In [4]:
# real data
data = np.loadtxt("data/rho20x40_SIMP_Edge.txt", dtype=np.float32)
# [bs, 1, 20, 40]
data = data.reshape(-1,1,40,20).transpose([0,1,3,2])

data_u = np.loadtxt("data/dis20x40_SIMP_Edge.txt", dtype=np.float32)
data_s = np.loadtxt("data/stress20x40_SIMP_Edge.txt", dtype=np.float32)

ref_u0 = torch.from_numpy(data_u).unsqueeze(1).to(device)
ref_uy = ref_u0[:,:,range(1,1722,2)]
ref_ux = ref_u0[:,:,range(0,1722,2)]
ref_s0 = torch.from_numpy(data_s).unsqueeze(1).to(device)
ref_sx = ref_s0[:,:,range(0,2583,3)]
ref_sy = ref_s0[:,:,range(1,2583,3)]
ref_sxy = ref_s0[:,:,range(2,2583,3)]
# [bs, 5, 21, 41]
ref = torch.cat([ref_ux, ref_uy, ref_sx, ref_sy, ref_sxy],1).view(-1,5,41,21).permute(0,1,3,2)
print(data.shape)

(1385, 1, 20, 40)


In [5]:
print(ref.shape)

torch.Size([1385, 5, 21, 41])


In [21]:
# 取部分
#input = torch.from_numpy(data[:7])
#output = ref[:7]
input = torch.from_numpy(data[:16,:,:,:])
output = ref[:16,:,:,:]

print(input.shape)
print(output.shape)

torch.Size([16, 1, 20, 40])
torch.Size([16, 5, 21, 41])


In [22]:
loss_pde = cc1_new(input, output, device)
loss_boundary = bc_new(output)

In [23]:
print(loss_pde)

tensor(0.0003, device='cuda:0')


In [13]:
print(loss_boundary)

tensor(0., device='cuda:0')


In [14]:
ux = output[:, [0]]
uy = output[:, [1]]

In [17]:
print(uy[0])

tensor([[[    0.0000,   412.0984,   742.3829,   986.8425,  1164.5623,
           1294.9640,  1390.6514,  1461.6199,  1515.1202,  1556.3571,
           1588.9587,  1615.3948,  1637.3041,  1655.7496,  1671.4106,
           1684.7227,  1695.9747,  1705.3754,  1713.0948,  1719.2874,
           1724.1056,  1727.7032,  1730.2360,  1731.8584,  1732.7209,
           1732.9663,  1732.7266,  1732.1216,  1731.2574,  1730.2269,
           1729.1093,  1727.9717,  1726.8711,  1725.8553,  1724.9642,
           1724.2312,  1723.6825,  1723.3335,  1723.1813,  1723.1879,
           1723.2625],
         [    0.0000,   288.6336,   583.4566,   810.7806,   983.2959,
           1111.8000,  1207.7457,  1279.9276,  1335.0966,  1378.1504,
           1412.5422,  1440.6351,  1464.0070,  1483.6904,  1500.3562,
           1514.4460,  1526.2657,  1536.0471,  1543.9863,  1550.2655,
           1555.0638,  1558.5601,  1560.9323,  1562.3542,  1562.9927,
           1563.0035,  1562.5291,  1561.6964,  1560.6176,  1559.388

## old loss test

In [9]:
# post output
WEIGHTS_2x2 = torch.FloatTensor( np.ones([1,1,2,2])/4 ).to(device)
o0 = F.conv2d(output[:,[0]], WEIGHTS_2x2, stride=1, padding=0, bias=None)
o1 = F.conv2d(output[:,[1]], WEIGHTS_2x2, stride=1, padding=0, bias=None) 
o2 = F.conv2d(output[:,[2]], WEIGHTS_2x2, stride=1, padding=0, bias=None) 
o3 = F.conv2d(output[:,[3]], WEIGHTS_2x2, stride=1, padding=0, bias=None) 
o4 = F.conv2d(output[:,[4]], WEIGHTS_2x2, stride=1, padding=0, bias=None) 
output_post = torch.cat([o0,o1,o2,o3,o4],1)
print(output_post.shape)

torch.Size([1, 5, 20, 40])


In [7]:
sobel_filter = SobelFilter(64, correct=False, device=device)

In [10]:
print (cc1(input, output, output_post, sobel_filter, device))

tensor(0.0002, device='cuda:0')


In [11]:
from models.darcy import cc2_fe
print (cc2_fe(input, output, device))

torch.Size([1, 800, 3, 1])


In [12]:
print(cc2new(output, sobel_filter))

tensor(1.4489, device='cuda:0')


In [13]:
print(cc2(output, sobel_filter))

tensor(5.0882, device='cuda:0')


In [14]:
loss_boundary = bc(output, output_post)
print(loss_boundary)

tensor(0.0392, device='cuda:0')


## loss detail

In [ ]:
# loss_boundary detail
ux = output[:, [0]]
uy = output[:, [1]]
lu = ux[:,:,:,0]**2 + uy[:,:,:,0]**2
sx = output[:, [2]]
sy = output[:, [3]]
sxy = output[:, [4]]
lbr = (sx[:,:,:,40]-1)**2 + (sxy[:,:,:,40])**2
lbt = (sy[:,:,0,2:-2])**2 + (sxy[:,:,0,2:-2])**2
lbb = (sy[:,:,20,2:-2])**2 + (sxy[:,:,20,2:-2])**2

In [ ]:
print(lu.sum([-1]).mean())
print(lbr.sum([-1]).mean())
print(lbt.sum([-1]).mean())
print(lbb.sum([-1]).mean())
# print(lbt.sum())
# print(lbb.sum())

In [ ]:
torch.cat([lu,lbb,lbt,lbr],2).sum([-1]).mean()

In [8]:
# cc1 detail
E = 1.0
nu = 0.3
# C0np = np.array()
C0 = E/(1-nu**2)*torch.Tensor([[1,nu,0],[nu,1,0],[0,0,(1-nu)/2]]).to(device)
pp = input.contiguous().view(input.shape[0], -1, 1, 1).to(device)
# pp = input.permute(0,1,3,2).contiguous().view(input.shape[0], -1, 1, 1).to(device)
C = pp**3*C0

# duxdx = sobel_filter.grad_h(output[:, [0]])
# duxdy = sobel_filter.grad_v(output[:, [0]])
# duydx = sobel_filter.grad_h(output[:, [1]])
# duydy = sobel_filter.grad_v(output[:, [1]])
# d1 = duxdx
# d2 = duydy
# d3 = duxdy+duydx
# du = torch.cat([d1,d2,d3],1)
# du_post = du.view(du.shape[0],3,-1,1).permute(0,2,1,3)
B=np.zeros([4,3,8],dtype=np.float32)
B[0,:,:]=np.array([ [-1,  0, 1, 0, 0, 0, 0, 0],
                    [0, -1, 0, 0, 0, 0, 0, 1],
                    [-1, -1, 0, 1, 0, 0, 1, 0] ])
B[1,:,:]=np.array([ [-1,  0,  1,  0, 0, 0, 0, 0],
                    [0,  0,  0, -1, 0, 1, 0, 0],
                    [0, -1, -1,  1, 1, 0, 0, 0] ])
B[2,:,:]=np.array([ [0, 0,  0,  0, 1, 0, -1,  0],
                    [0, 0,  0, -1, 0, 1,  0,  0],
                    [0, 0, -1,  0, 1, 1,  0, -1] ])
B[3,:,:]=np.array([ [0,  0, 0, 0, 1, 0, -1,  0],
                    [0, -1, 0, 0, 0, 0,  0,  1],
                    [-1,  0, 0, 0, 0, 1,  1, -1] ])
B = torch.from_numpy(B).to(device)

B0 = torch.Tensor([[-0.5,0,-0.5],[0,-0.5,-0.5],[0.5,0,-0.5],\
    [0,-0.5,0.5],[0.5,0,0.5] ,[0,0.5,0.5],[-0.5,0,0.5],[0,-0.5,-0.5]]).to(device)
B0T = B0.transpose(0,1)
ux = output[:,[0]]
uy = output[:,[1]]

# k1 = torch.FloatTensor( np.array([[[[-0.5,0.5],[-0.5,0.5]]]]) ).to(device)
# k2 = torch.FloatTensor( np.array([[[[-0.5,-0.5],[-0.5,0.5]]]]) ).to(device)
# k31 = torch.FloatTensor( np.array([[[[-0.5,-0.5],[0.5,0.5]]]]) ).to(device)
# k32 = torch.FloatTensor( np.array([[[[-0.5,0.5],[-0.5,0.5]]]]) ).to(device)
# ux1 = F.conv2d(ux, k1, stride=1, padding=0, bias=None)
# uy1 = F.conv2d(uy, k2, stride=1, padding=0, bias=None)
# ux2 = F.conv2d(ux, k31, stride=1, padding=0, bias=None)
# uy2 = F.conv2d(uy, k32, stride=1, padding=0, bias=None)
# utest = torch.cat([ux1, uy1, ux2+uy2], 1)

unfold = nn.Unfold(kernel_size=(2, 2))
# [bs, 4, w*h]
uex = unfold(ux)
uey = unfold(uy)
ue_not = torch.cat([uex, uey], 1).permute([0,2,1])
ue = ue_not[:,:,[0,4,1,5,3,7,2,6]].unsqueeze(3)



# sig = output_post[:, [2,3,4]]
# sig_post = sig.view(sig.shape[0],3,-1,1).permute(0,2,1,3)
du0 = torch.matmul(C0@B[0,:,:],ue)
du1 = torch.matmul(C0@B[1,:,:],ue)
du2 = torch.matmul(C0@B[2,:,:],ue)
du3 = torch.matmul(C0@B[3,:,:],ue)
du0t = du0.permute([0,2,1,3]).contiguous().view(-1,3,20,40)
du1t = du1.permute([0,2,1,3]).contiguous().view(-1,3,20,40)
du2t = du2.permute([0,2,1,3]).contiguous().view(-1,3,20,40)
du3t = du3.permute([0,2,1,3]).contiguous().view(-1,3,20,40)
ones = torch.ones_like(du0t)

masks =torch.zeros([du0.shape[0],3,21,41]).to(device)
result =torch.zeros([du0.shape[0],3,21,41]).to(device)

penal=3
Dp=torch.zeros([du0.shape[0],3,20,40]).to(device)
Dp[0,0,:,:]=input**penal
Dp[0,1,:,:]=input**penal
Dp[0,2,:,:]=input**penal

result[:,:,:20,:40] = du0t*Dp
result[:,:,:20,1:] = result[:,:,:20,1:]+du1t*Dp
result[:,:,1:,:40] = result[:,:,1:,:40]+du3t*Dp
result[:,:,1:,1:] = result[:,:,1:,1:]+du2t*Dp



masks[:,:,:20,:40] += ones
masks[:,:,:20,1:] += ones
masks[:,:,1:,:40] += ones
masks[:,:,1:,1:] += ones
lp1 = (result / masks) - output[:, [2,3,4]]

RuntimeError: expand(torch.FloatTensor{[8, 1, 20, 40]}, size=[20, 40]): the number of sizes provided (2) must be greater or equal to the number of dimensions in the tensor (4)

In [25]:
E = 1.0
nu = 0.3
# C0np = np.array()
C0 = E/(1-nu**2)*torch.Tensor([[1,nu,0],[nu,1,0],[0,0,(1-nu)/2]]).to(device)
pp = input.contiguous().view(input.shape[0], -1, 1, 1).to(device)
# pp = input.permute(0,1,3,2).contiguous().view(input.shape[0], -1, 1, 1).to(device)
#print(['shape of pp:',pp.shape])
#print(input[0,0,0,:])
#print(pp[0,0:41,0,0])
C = pp**3*C0
        
ux = output[:,[0]]
uy = output[:,[1]]
unfold = nn.Unfold(kernel_size=(2, 2))
# [bs, 4, w*h]
uex = unfold(ux)
uey = unfold(uy)
ue_not = torch.cat([uex, uey], 1).permute([0,2,1])
ue = ue_not[:,:,[0,4,1,5,3,7,2,6]].unsqueeze(3)

Bxy=np.zeros([2,3,8],dtype=np.float32)
Bxy[0,:,:]=np.array([ [0, 0,  0,  0, 0, 0,  0,  0],
                   [0, 1,  0, -1, 0, 1,  0, -1],
                   [1, 0, -1,  0, 1, 0, -1,  0] ])
Bxy[1,:,:]=np.array([ [1, 0, -1,  0, 1, 0, -1,  0],
                   [0, 0,  0,  0, 0, 0,  0,  0],
                   [0, 1,  0, -1, 0, 1,  0, -1] ])
Bxy = torch.from_numpy(Bxy).to(device)
S_x = torch.matmul(C, torch.matmul(Bxy[0,:,:],ue))
S_y = torch.matmul(C, torch.matmul(Bxy[1,:,:],ue))#torch.Size([8, 800, 3, 1])
dsxdx = S_x[:,:,0,:]
dsxydx = S_x[:,:,2,:]

dsydy = S_y[:,:,1,:]
dsxydy = S_y[:,:,2,:]

ds = torch.cat([dsxdx+dsxydy, dsydy+dsxydx],2)
#dsxdx+dsxydy, dsydy+dsxydx
print(['shape of ue:',ue.shape])
print(dsxdx.shape)
#print(dsxdx[0,0:40,0])
#print(dsxydy[0,0:40,0])

#print(ds.shape)
#print(ds[0,0:42,0])
print((ds[0,:,0] ** 2).sum())
print((ds[0,:,1] ** 2).sum())
#print((ds[0,:,:] ** 2).sum())
print((ds ** 2).sum([1,2]))
print(dsxydy[0,0:40,0])
#print(dsydy[7,40:80,0])
ppp=(dsydy[0,:,0]).reshape([20,40])
#print(ppp.shape)
print(ppp)

['shape of ue:', torch.Size([1, 800, 8, 1])]
torch.Size([1, 800, 1])
tensor(22.1057, device='cuda:0')
tensor(677.5823, device='cuda:0')
tensor([699.6879], device='cuda:0')
tensor([-3.8469e-01, -2.2146e-01,  1.2220e-01,  2.4472e-03, -8.8695e-05,
        -1.6975e-05, -3.2478e-06, -3.6357e-07,  2.4155e-06,  2.3503e-06,
        -4.0335e-06, -1.5748e-05, -2.7668e-05, -3.7400e-05, -4.4582e-05,
        -4.6272e-05, -3.7589e-05, -1.5593e-05,  1.6767e-05,  5.0770e-05,
         7.1423e-05,  6.1259e-05,  4.4929e-06, -1.0816e-04, -2.7593e-04,
        -4.8702e-04, -7.2155e-04, -9.5177e-04, -1.1636e-03, -1.3308e-03,
        -1.4535e-03, -1.5323e-03, -1.5808e-03, -1.6140e-03, -1.6655e-03,
        -1.7635e-03, -1.9210e-03, -2.1030e-03, -1.2894e-03,  5.3784e-03],
       device='cuda:0')
tensor([[ 5.6373e+00,  1.9038e+00,  9.8449e-02, -1.8375e-04, -7.2078e-05,
         -3.0276e-05, -2.6023e-05, -2.6184e-05, -2.7259e-05, -3.3266e-05,
         -4.3944e-05, -5.2178e-05, -5.4006e-05, -4.6991e-05, -2.4379e-0

In [ ]:
print(C.shape)

In [ ]:
print(ue[0,0,:,0])
ue[0,1,:,0]

In [ ]:
print(du0t.shape)
du0t[0,0,0,0]

In [ ]:
print(lp1.shape)
print((lp1**2).sum([1,2,3]).mean())

In [ ]:
sxy[0,0,:,0]

In [ ]:
print(masks.shape)
masks[0,1,:,:]

In [55]:
# cc2 loss detail
dsxdx = sobel_filter.grad_h(output[:, [2]])
dsydy = sobel_filter.grad_v(output[:, [3]])
dsxydx = sobel_filter.grad_h(output[:, [4]])
dsxydy = sobel_filter.grad_v(output[:, [4]])

ds = torch.cat([dsxdx+dsxydy, dsydy+dsxydx],1)
print((ds ** 2).sum([1,2,3]).mean())

print(ds[0,0,:,0])
print(ds[0,0,0,:])
print((ds[0,:,:,:] ** 2).sum())
print((ds ** 2).sum([1,2,3]))
print(ds[0,0,:,:])

tensor(22.9232, device='cuda:0')
tensor([0.6787, 0.5282, 0.4706, 0.4518, 0.4471, 0.4436, 0.4421, 0.4412, 0.4407,
        0.4405, 0.4405, 0.4405, 0.4407, 0.4412, 0.4421, 0.4436, 0.4471, 0.4518,
        0.4706, 0.5282, 0.6787], device='cuda:0')
tensor([ 6.7867e-01, -9.7750e-02, -7.7887e-02, -3.1612e-02, -1.4863e-02,
        -6.9625e-03, -2.7250e-03, -4.3751e-04,  8.3749e-04,  1.5125e-03,
         1.8250e-03,  1.9375e-03,  1.9625e-03,  1.9625e-03,  1.8875e-03,
         1.8000e-03,  1.7125e-03,  1.6250e-03,  1.5125e-03,  1.3875e-03,
         1.2875e-03,  1.1750e-03,  1.0625e-03,  9.4999e-04,  8.6249e-04,
         7.6250e-04,  6.7500e-04,  5.8750e-04,  4.8751e-04,  4.1249e-04,
         3.7499e-04,  3.2501e-04,  2.3750e-04,  1.9998e-04,  1.7501e-04,
         1.3751e-04,  8.7495e-05,  2.4998e-05,  8.4765e-10,  1.2497e-05,
        -3.7499e-01], device='cuda:0')
tensor(10.1765, device='cuda:0')
tensor([10.1765, 10.4226, 11.0716, 12.5189, 16.1531, 24.3107, 41.2842, 57.4478],
       device='cuda:

In [14]:
print(dsxdx.shape)
print(ds.shape)

torch.Size([8, 1, 21, 41])
torch.Size([8, 2, 21, 41])


In [12]:
print((ds ** 2).sum([1,2,3]).mean())

tensor(13.6358, device='cuda:0')


In [25]:
dsxdx.mean([0])

tensor([[[ 4.7309e-01, -9.8053e-02, -7.7725e-02, -5.3725e-02, -3.4291e-02,
          -2.0941e-02, -1.2106e-02, -6.2938e-03, -2.4844e-03,  1.4901e-08,
           1.6156e-03,  2.6500e-03,  3.2969e-03,  3.6719e-03,  3.8469e-03,
           3.8843e-03,  3.8281e-03,  3.6969e-03,  3.5125e-03,  3.3093e-03,
           3.0781e-03,  2.8157e-03,  2.5468e-03,  2.2875e-03,  2.0344e-03,
           1.7906e-03,  1.5719e-03,  1.3594e-03,  1.1531e-03,  9.8120e-04,
           8.3748e-04,  6.9065e-04,  5.5625e-04,  4.5626e-04,  3.7191e-04,
           2.9064e-04,  1.9682e-04,  1.0624e-04,  5.9374e-05, -3.7496e-01],
         [ 5.7779e-01, -3.6600e-02, -4.0275e-02, -4.2078e-02, -3.5528e-02,
          -2.6706e-02, -1.8759e-02, -1.2434e-02, -7.6781e-03, -4.1750e-03,
          -1.6031e-03,  2.5000e-04,  1.5594e-03,  2.4750e-03,  3.0875e-03,
           3.4500e-03,  3.6343e-03,  3.6844e-03,  3.6250e-03,  3.5094e-03,
           3.3344e-03,  3.0938e-03,  2.8281e-03,  2.5719e-03,  2.3188e-03,
           2.0531e-03,  